- choropleth map for all the teams and their favorite (worst) ground
- best openers and their help in win
- best finishers and their help in win
- bowlers and batsman pair to watch
- map of most destructive batsman
- out of 100 balls what happens on them

In [143]:
!pip install matplotlib

    2% |▋                               | 276kB 15kB/s eta 0:14:44Exception:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pip/_vendor/requests/packages/urllib3/response.py", line 232, in _error_catcher
    yield
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pip/_vendor/requests/packages/urllib3/response.py", line 314, in read
    data = self._fp.read(amt)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pip/_vendor/cachecontrol/filewrapper.py", line 60, in read
    data = self.__fp.read(amt)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 449, in read
    n = self.readinto(b)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 493, in readinto
    n = self.fp.readinto(b)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socket.

In [33]:
import pandas as pd
from collections import Counter

In [34]:
df = pd.read_csv('./datasets/matches.dmp')

In [35]:
df.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2008,Bangalore,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Kolkata Knight Riders,140,0,BB McCullum,M Chinnaswamy Stadium,Asad Rauf,RE Koertzen,NaN
1,2,2008,Chandigarh,2008-04-19,Chennai Super Kings,Kings XI Punjab,Chennai Super Kings,bat,normal,0,Chennai Super Kings,33,0,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",MR Benson,SL Shastri,NaN
2,3,2008,Delhi,2008-04-19,Rajasthan Royals,Delhi Daredevils,Rajasthan Royals,bat,normal,0,Delhi Daredevils,0,9,MF Maharoof,Feroz Shah Kotla,Aleem Dar,GA Pratapkumar,NaN
3,4,2008,Mumbai,2008-04-20,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,normal,0,Royal Challengers Bangalore,0,5,MV Boucher,Wankhede Stadium,SJ Davis,DJ Harper,NaN
4,5,2008,Kolkata,2008-04-20,Deccan Chargers,Kolkata Knight Riders,Deccan Chargers,bat,normal,0,Kolkata Knight Riders,0,5,DJ Hussey,Eden Gardens,BF Bowden,K Hariharan,NaN


### Teams

In [36]:
teams = list(df['team1'].unique())
teams

['Kolkata Knight Riders',
 'Chennai Super Kings',
 'Rajasthan Royals',
 'Mumbai Indians',
 'Deccan Chargers',
 'Kings XI Punjab',
 'Royal Challengers Bangalore',
 'Delhi Daredevils',
 'Kochi Tuskers Kerala',
 'Pune Warriors',
 'Sunrisers Hyderabad',
 'Rising Pune Supergiants',
 'Gujarat Lions']

### Ground Map Processing

### Grounds

In [37]:
grounds = list(df['venue'].unique())
grounds

['M Chinnaswamy Stadium',
 'Punjab Cricket Association Stadium, Mohali',
 'Feroz Shah Kotla',
 'Wankhede Stadium',
 'Eden Gardens',
 'Sawai Mansingh Stadium',
 'Rajiv Gandhi International Stadium, Uppal',
 'MA Chidambaram Stadium, Chepauk',
 'Dr DY Patil Sports Academy',
 'Newlands',
 "St George's Park",
 'Kingsmead',
 'SuperSport Park',
 'Buffalo Park',
 'New Wanderers Stadium',
 'De Beers Diamond Oval',
 'OUTsurance Oval',
 'Brabourne Stadium',
 'Sardar Patel Stadium, Motera',
 'Barabati Stadium',
 'Vidarbha Cricket Association Stadium, Jamtha',
 'Himachal Pradesh Cricket Association Stadium',
 'Nehru Stadium',
 'Holkar Cricket Stadium',
 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
 'Subrata Roy Sahara Stadium',
 'Shaheed Veer Narayan Singh International Stadium',
 'JSCA International Stadium Complex',
 'Sheikh Zayed Stadium',
 'Sharjah Cricket Stadium',
 'Dubai International Cricket Stadium',
 'Maharashtra Cricket Association Stadium',
 'Punjab Cricket Association IS Bind

In [38]:
non_indian_grounds = [
     'Newlands',
     "St George's Park",
     'Kingsmead',
     'SuperSport Park',
     'Buffalo Park',
     'New Wanderers Stadium',
     'De Beers Diamond Oval',
     'OUTsurance Oval',
     'Sheikh Zayed Stadium',
     'Sharjah Cricket Stadium',
     'Dubai International Cricket Stadium',
]

def is_indian_ground(ground_name):
    return False if ground_name in non_indian_grounds else True

In [39]:
def get_grounds_results(team_name):
    # append the team1/team2 appearance
    df1 = df[df['team1'] == team_name]
    df2 = df[df['team2'] == team_name]
    _df = df1.append(df2)
    
    # win and lose for each ground
    _won = Counter(filter(lambda g: is_indian_ground(g), list(_df[_df['winner'] == team_name]['venue'])))
    _lost = Counter(filter(lambda g: is_indian_ground(g), list(_df[_df['winner'] != team_name]['venue'])))
    
    # total grounds the team has played on
    _grounds = set(_won).union(set(_lost))
    
    _matches = []
    
    # Mohali ground has multiple names in dataset
    mohali_v1_name = "Punjab Cricket Association Stadium, Mohali"
    mohali_v2_name = "Punjab Cricket Association IS Bindra Stadium, Mohali"
    mohali_v2_won, mohali_v2_lost = _won.get(mohali_v2_name, 0), _lost.get(mohali_v2_name, 0)
    
    # iterate over all grounds
    for g in list(_grounds):
        # ignore the mohali grounds
        if g not in [mohali_v1_name, mohali_v2_name]:
            _matches.append({
                'ground': g,
                'won': _won.get(g, 0),
                'lost': _lost.get(g, 0),
            })  
            
    # unify the mohali results
    _matches.append({
        'ground': mohali_v1_name,
        'won': _won.get(mohali_v1_name, 0) + (mohali_v2_won),
        'lost': _lost.get(mohali_v1_name, 0) + (mohali_v2_lost),
    })
    
    result = {
        team_name: _matches
    }
    
    return result

In [40]:
for t in teams:
    print('--------------------\n', t)
    print(get_grounds_results(t))

--------------------
 Kolkata Knight Riders
{'Kolkata Knight Riders': [{'ground': 'Maharashtra Cricket Association Stadium', 'won': 2, 'lost': 0}, {'ground': 'Sardar Patel Stadium, Motera', 'won': 0, 'lost': 2}, {'ground': 'Nehru Stadium', 'won': 0, 'lost': 1}, {'ground': 'Dr DY Patil Sports Academy', 'won': 2, 'lost': 0}, {'ground': 'Sawai Mansingh Stadium', 'won': 1, 'lost': 3}, {'ground': 'Shaheed Veer Narayan Singh International Stadium', 'won': 0, 'lost': 1}, {'ground': 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium', 'won': 0, 'lost': 1}, {'ground': 'MA Chidambaram Stadium, Chepauk', 'won': 2, 'lost': 5}, {'ground': 'Rajiv Gandhi International Stadium, Uppal', 'won': 4, 'lost': 1}, {'ground': 'JSCA International Stadium Complex', 'won': 1, 'lost': 2}, {'ground': 'Brabourne Stadium', 'won': 0, 'lost': 2}, {'ground': 'Eden Gardens', 'won': 33, 'lost': 18}, {'ground': 'Feroz Shah Kotla', 'won': 4, 'lost': 3}, {'ground': 'M Chinnaswamy Stadium', 'won': 4, 'lost': 4}, {'ground':

### Toss Effect Processing

In [41]:
df.shape

(577, 18)

In [42]:
df[df['toss_winner'] == df['winner']].shape

(291, 18)

In [43]:
def get_toss_winning_percentage(team):
    # matches
    _df = df[df['team1'] == team].append(df[df['team2'] == team])
    total_matches = _df.shape[0]
    
    # toss-won matches
    toss_won_df = _df[_df['toss_winner'] == team]
    # toss-lost matches
    toss_lost_df = _df[_df['toss_winner'] != team]
    
    # toss-won match-won
    toss_won_match_won_df = toss_won_df[toss_won_df['winner'] == team]
    # toss-won match-lost
    toss_won_match_lost_df = toss_won_df[toss_won_df['winner'] != team]
    
    # toss-lost match-won
    toss_lost_match_won_df = toss_lost_df[toss_lost_df['winner'] == team]
    # toss-lost match-lost
    toss_lost_match_lost_df = toss_lost_df[toss_lost_df['winner'] != team]
    
    return {
        "team": team,
        "total_match": total_matches,
        "won_toss": toss_won_df.shape[0],
        "won_toss_won_match": toss_won_match_won_df.shape[0],
        "won_toss_lost_match": toss_won_match_lost_df.shape[0],
        "lost_toss": toss_lost_df.shape[0],
        "lost_toss_won_match": toss_lost_match_won_df.shape[0],
        "lost_toss_lost_match": toss_lost_match_lost_df.shape[0]
    }

In [44]:
for team in teams:
    print(get_toss_winning_percentage(team))

{'team': 'Kolkata Knight Riders', 'total_match': 132, 'won_toss': 69, 'won_toss_won_match': 38, 'won_toss_lost_match': 31, 'lost_toss': 63, 'lost_toss_won_match': 30, 'lost_toss_lost_match': 33}
{'team': 'Chennai Super Kings', 'total_match': 131, 'won_toss': 66, 'won_toss_won_match': 42, 'won_toss_lost_match': 24, 'lost_toss': 65, 'lost_toss_won_match': 37, 'lost_toss_lost_match': 28}
{'team': 'Rajasthan Royals', 'total_match': 118, 'won_toss': 63, 'won_toss_won_match': 34, 'won_toss_lost_match': 29, 'lost_toss': 55, 'lost_toss_won_match': 29, 'lost_toss_lost_match': 26}
{'team': 'Mumbai Indians', 'total_match': 140, 'won_toss': 74, 'won_toss_won_match': 41, 'won_toss_lost_match': 33, 'lost_toss': 66, 'lost_toss_won_match': 39, 'lost_toss_lost_match': 27}
{'team': 'Deccan Chargers', 'total_match': 75, 'won_toss': 43, 'won_toss_won_match': 19, 'won_toss_lost_match': 24, 'lost_toss': 32, 'lost_toss_won_match': 10, 'lost_toss_lost_match': 22}
{'team': 'Kings XI Punjab', 'total_match': 134

### Slope Chart Processing

In [45]:
_df = df[df['season'] == 2008]
set(_df['team1'].append(_df['team2']))

{'Chennai Super Kings',
 'Deccan Chargers',
 'Delhi Daredevils',
 'Kings XI Punjab',
 'Kolkata Knight Riders',
 'Mumbai Indians',
 'Rajasthan Royals',
 'Royal Challengers Bangalore'}

In [46]:
from math import floor
from collections import defaultdict

def get_win_loss_ratio(team, year):
    _df = df[df['season'] == year]
    _df1 = _df[_df['team1'] == team]
    _df2 = _df[_df['team2'] == team]
    _df = _df1.append(_df2)
    
    try:
        return floor((_df[_df['winner'] == team].shape[0] / _df.shape[0]) * 100)
    except ZeroDivisionError:
        return 0

def get_complete_win_loss_ratio():
    result = defaultdict(dict)
    for year in range(2008, 2017):
        _df = df[df['season'] == year]
        for team in teams:
            if team in set(_df['team1'].append(_df['team2'])):
                result[year][team] = get_win_loss_ratio(team, year)
            
    return result

dict(get_complete_win_loss_ratio())

{2008: {'Chennai Super Kings': 56,
  'Deccan Chargers': 14,
  'Delhi Daredevils': 50,
  'Kings XI Punjab': 66,
  'Kolkata Knight Riders': 46,
  'Mumbai Indians': 50,
  'Rajasthan Royals': 81,
  'Royal Challengers Bangalore': 28},
 2009: {'Chennai Super Kings': 57,
  'Deccan Chargers': 56,
  'Delhi Daredevils': 66,
  'Kings XI Punjab': 50,
  'Kolkata Knight Riders': 23,
  'Mumbai Indians': 38,
  'Rajasthan Royals': 46,
  'Royal Challengers Bangalore': 56},
 2010: {'Chennai Super Kings': 56,
  'Deccan Chargers': 50,
  'Delhi Daredevils': 50,
  'Kings XI Punjab': 28,
  'Kolkata Knight Riders': 50,
  'Mumbai Indians': 68,
  'Rajasthan Royals': 42,
  'Royal Challengers Bangalore': 50},
 2011: {'Chennai Super Kings': 68,
  'Deccan Chargers': 42,
  'Delhi Daredevils': 28,
  'Kings XI Punjab': 50,
  'Kochi Tuskers Kerala': 42,
  'Kolkata Knight Riders': 53,
  'Mumbai Indians': 62,
  'Pune Warriors': 28,
  'Rajasthan Royals': 46,
  'Royal Challengers Bangalore': 62},
 2012: {'Chennai Super King

In [47]:
delivery_df = pd.read_csv('./datasets/deliveries.dmp')

In [48]:
def get_average_score(team, year):
    _df = df[df['season'] == year]
    _df1 = _df[_df['team1'] == team]
    _df2 = _df[_df['team2'] == team]
    _df = _df1.append(_df2)
    match_ids = list(_df['id'])
    
    _scores = []
    for mid in match_ids:
        _ddf = delivery_df[delivery_df['match_id'] == mid]
        _scores.append(sum(_ddf[_ddf['batting_team'] == team]['total_runs']))
        
    return floor(sum(_scores) / len(_scores))

def get_complete_average_score():
    result = defaultdict(dict)
    for year in range(2008, 2017):
        _df = df[df['season'] == year]
        for team in teams:
            if team in set(_df['team1'].append(_df['team2'])):
                result[year][team] = get_average_score(team, year)
            
    return result

dict(get_complete_average_score())

{2008: {'Chennai Super Kings': 157,
  'Deccan Chargers': 159,
  'Delhi Daredevils': 151,
  'Kings XI Punjab': 164,
  'Kolkata Knight Riders': 149,
  'Mumbai Indians': 148,
  'Rajasthan Royals': 162,
  'Royal Challengers Bangalore': 141},
 2009: {'Chennai Super Kings': 159,
  'Deccan Chargers': 150,
  'Delhi Daredevils': 142,
  'Kings XI Punjab': 137,
  'Kolkata Knight Riders': 136,
  'Mumbai Indians': 145,
  'Rajasthan Royals': 131,
  'Royal Challengers Bangalore': 142},
 2010: {'Chennai Super Kings': 162,
  'Deccan Chargers': 148,
  'Delhi Daredevils': 153,
  'Kings XI Punjab': 163,
  'Kolkata Knight Riders': 153,
  'Mumbai Indians': 171,
  'Rajasthan Royals': 155,
  'Royal Challengers Bangalore': 150},
 2011: {'Chennai Super Kings': 160,
  'Deccan Chargers': 152,
  'Delhi Daredevils': 149,
  'Kings XI Punjab': 158,
  'Kochi Tuskers Kerala': 135,
  'Kolkata Knight Riders': 134,
  'Mumbai Indians': 143,
  'Pune Warriors': 126,
  'Rajasthan Royals': 129,
  'Royal Challengers Bangalore':

In [49]:
get_average_score('Rajasthan Royals', 2008)

162

In [50]:
delivery_df.shape

(136598, 21)

In [51]:
delivery_df.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')

In [52]:
delivery_df[delivery_df['batting_team'] == 'Rajasthan Royals'].tail(20)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
121989,515,2,Rajasthan Royals,Royal Challengers Bangalore,16,6,CH Morris,DJ Hooda,MA Starc,0,...,0,0,0,0,0,0,0,CH Morris,caught,YS Chahal
121990,515,2,Rajasthan Royals,Royal Challengers Bangalore,17,1,DJ Hooda,Ankit Sharma,D Wiese,0,...,0,0,0,0,2,0,2,NaN,NaN,NaN
121991,515,2,Rajasthan Royals,Royal Challengers Bangalore,17,2,DJ Hooda,Ankit Sharma,D Wiese,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
121992,515,2,Rajasthan Royals,Royal Challengers Bangalore,17,3,DJ Hooda,Ankit Sharma,D Wiese,0,...,0,0,0,0,1,0,1,NaN,NaN,NaN
121993,515,2,Rajasthan Royals,Royal Challengers Bangalore,17,4,Ankit Sharma,DJ Hooda,D Wiese,0,...,0,0,0,0,1,0,1,NaN,NaN,NaN
121994,515,2,Rajasthan Royals,Royal Challengers Bangalore,17,5,DJ Hooda,Ankit Sharma,D Wiese,0,...,0,0,0,0,0,0,0,DJ Hooda,caught,MA Starc
121995,515,2,Rajasthan Royals,Royal Challengers Bangalore,17,6,Ankit Sharma,DS Kulkarni,D Wiese,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
121996,515,2,Rajasthan Royals,Royal Challengers Bangalore,18,1,DS Kulkarni,Ankit Sharma,MA Starc,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
121997,515,2,Rajasthan Royals,Royal Challengers Bangalore,18,2,DS Kulkarni,Ankit Sharma,MA Starc,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
121998,515,2,Rajasthan Royals,Royal Challengers Bangalore,18,3,DS Kulkarni,Ankit Sharma,MA Starc,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN


### Most Dismissals

In [53]:
dismissed_df = delivery_df[delivery_df['player_dismissed'].notnull()]

batsman_bowler_pairs = Counter()
for index, row in dismissed_df[['bowler', 'player_dismissed']].iterrows():
    batsman_bowler_pairs['{}:{}'.format(row['bowler'], row['player_dismissed'])] += 1

In [54]:
most_dismissed_pairs = batsman_bowler_pairs.most_common(30)
result = []

for p in most_dismissed_pairs:
    bowler, batsman = p[0].split(':')
    result.append({
        'source': bowler,
        'target': batsman,
        'value': p[1]
    })
    
print('bowlers', set(map(lambda r: r['source'], result)))
print('\n')
print('batsmen', set(map(lambda r: r['target'], result)))
print('\n')
print(result)
print('\n')
print(set(map(lambda r: r['source'], result)).intersection(set(map(lambda r: r['target'], result))))

bowlers {'B Kumar', 'PP Chawla', 'MM Sharma', 'P Kumar', 'Z Khan', 'DJ Bravo', 'R Sharma', 'PP Ojha', 'IK Pathan', 'A Nehra', 'A Mishra', 'RP Singh', 'Harbhajan Singh', 'RA Jadeja', 'R Vinay Kumar', 'SK Trivedi'}


batsmen {'SC Ganguly', 'R Dravid', 'MK Pandey', 'KC Sangakkara', 'NV Ojha', 'BB McCullum', 'RG Sharma', 'MS Dhoni', 'PA Patel', 'AB de Villiers', 'RV Uthappa', 'SK Raina', 'DJ Hussey', 'V Kohli', 'AC Gilchrist', 'JH Kallis', 'G Gambhir', 'AM Rahane', 'SR Watson', 'MEK Hussey', 'RA Jadeja', 'AT Rayudu'}


[{'source': 'Z Khan', 'target': 'MS Dhoni', 'value': 7}, {'source': 'PP Ojha', 'target': 'MS Dhoni', 'value': 7}, {'source': 'A Nehra', 'target': 'V Kohli', 'value': 6}, {'source': 'R Vinay Kumar', 'target': 'RG Sharma', 'value': 6}, {'source': 'B Kumar', 'target': 'AM Rahane', 'value': 6}, {'source': 'Z Khan', 'target': 'G Gambhir', 'value': 5}, {'source': 'SK Trivedi', 'target': 'RG Sharma', 'value': 5}, {'source': 'PP Ojha', 'target': 'G Gambhir', 'value': 5}, {'source': 

In [55]:
batsmen = {'G Gambhir', 'SC Ganguly', 'JH Kallis', 'R Dravid', 'DJ Hussey', 'KC Sangakkara', 'BB McCullum', 'MEK Hussey', 'MS Dhoni', 'AB de Villiers', 'SR Watson', 'PA Patel', 'AC Gilchrist', 'RV Uthappa', 'AM Rahane', 'SK Raina', 'RA Jadeja', 'V Kohli', 'RG Sharma', 'NV Ojha', 'MK Pandey', 'AT Rayudu'}

In [56]:
d = {}
for b in batsmen:
    d[b]='batsmen'

In [57]:
d

{'AB de Villiers': 'batsmen',
 'AC Gilchrist': 'batsmen',
 'AM Rahane': 'batsmen',
 'AT Rayudu': 'batsmen',
 'BB McCullum': 'batsmen',
 'DJ Hussey': 'batsmen',
 'G Gambhir': 'batsmen',
 'JH Kallis': 'batsmen',
 'KC Sangakkara': 'batsmen',
 'MEK Hussey': 'batsmen',
 'MK Pandey': 'batsmen',
 'MS Dhoni': 'batsmen',
 'NV Ojha': 'batsmen',
 'PA Patel': 'batsmen',
 'R Dravid': 'batsmen',
 'RA Jadeja': 'batsmen',
 'RG Sharma': 'batsmen',
 'RV Uthappa': 'batsmen',
 'SC Ganguly': 'batsmen',
 'SK Raina': 'batsmen',
 'SR Watson': 'batsmen',
 'V Kohli': 'batsmen'}

### Percentage Share (per ball)

In [174]:
from math import ceil

total_deliveries = delivery_df.shape[0]
total_deliveries

136598

In [175]:
wicket_deliveries = delivery_df[delivery_df['player_dismissed'].notnull()].shape[0]
print(wicket_deliveries)
print('wicket_deliveries', ceil(100*wicket_deliveries / total_deliveries))

6727
wicket_deliveries 5


In [176]:
six_deliveries = delivery_df[delivery_df['batsman_runs'] == 6].shape[0]
print('six_deliveries', ceil(100*six_deliveries / total_deliveries))
six_deliveries

six_deliveries 5


5817

In [177]:
four_deliveries = delivery_df[delivery_df['batsman_runs'] == 4].shape[0]
print('four_deliveries', ceil(100*four_deliveries / total_deliveries))
four_deliveries

four_deliveries 12


15421

In [178]:
extra_run_deliveries = delivery_df[delivery_df['extra_runs'] != 0].shape[0]
print('extra_run_deliveries', ceil(100*extra_run_deliveries / total_deliveries))
extra_run_deliveries

extra_run_deliveries 6


7469

In [63]:
one_deliveries = delivery_df[delivery_df['batsman_runs'] == 1].shape[0]
print('one_deliveries', ceil(100*one_deliveries / total_deliveries))

one_deliveries 37


In [64]:
one_deliveries = delivery_df[delivery_df['batsman_runs'] == 2].shape[0]
print('two_deliveries', ceil(100*one_deliveries / total_deliveries))

two_deliveries 7


In [65]:
one_deliveries = delivery_df[delivery_df['batsman_runs'] == 3].shape[0]
print('two_deliveries', ceil(100*one_deliveries / total_deliveries))

two_deliveries 1


### Network Graph based on wins

In [74]:
import networkx as nx

In [140]:
# possible match-pair teams
team_pairs = list(itertools.combinations(teams, 2))

def pair_selector_function(p, q, team_a, team_b):
    return sorted([p, q]) == sorted([team_a, team_b])

def get_pair_results(team_a, team_b):
    _df = df[df[['team1','team2']].apply(lambda x: pair_selector_function(*x, team_a, team_b), axis=1)]
    
    total = _df.shape[0]
    total_team_a_win = _df[_df['winner'] == team_a].shape[0]
    total_team_b_win = _df[_df['winner'] == team_b].shape[0]
    
    # print("Team A: {}, Team B: {}".format(team_a, team_b))
    
    return {
        'total': total,
        team_a: total_team_a_win,
        team_b: total_team_b_win,
    }

In [158]:
DG = nx.DiGraph()
weighted_edge_list = []

team_never_played = set()

for team_pair in team_pairs:
    pair_results = get_pair_results(team_pair[0], team_pair[1])
    
    try:
        weighted_edge_list.append((team_pair[0], team_pair[1], (pair_results[team_pair[0]] / pair_results['total'])))
    except:
        team_never_played.add(team_pair)
        
    try:
        weighted_edge_list.append((team_pair[1], team_pair[0], (pair_results[team_pair[1]] / pair_results['total'])))
    except:
        team_never_played.add(team_pair)
    
DG.add_weighted_edges_from(weighted_edge_list)

for t, s in nx.pagerank(DG).items():
    print(t, s*100)

<bound method Graph.edge_subgraph of <networkx.classes.digraph.DiGraph object at 0x1122b6898>>
Kolkata Knight Riders 9.050342854962597
Chennai Super Kings 6.95726173248562
Rajasthan Royals 7.027748244335999
Mumbai Indians 9.616531976554905
Deccan Chargers 7.366434739164067
Kings XI Punjab 8.825892275332073
Royal Challengers Bangalore 6.544641803806213
Delhi Daredevils 10.436575163917928
Kochi Tuskers Kerala 9.115591743152304
Pune Warriors 9.07750001452412
Sunrisers Hyderabad 6.231418288161537
Rising Pune Supergiants 5.705388213616469
Gujarat Lions 4.044672949986168


In [148]:
print(weighted_edge_list)

[('Kolkata Knight Riders', 'Chennai Super Kings', 0.375), ('Chennai Super Kings', 'Kolkata Knight Riders', 0.625), ('Kolkata Knight Riders', 'Rajasthan Royals', 0.4), ('Rajasthan Royals', 'Kolkata Knight Riders', 0.6), ('Kolkata Knight Riders', 'Mumbai Indians', 0.2777777777777778), ('Mumbai Indians', 'Kolkata Knight Riders', 0.7222222222222222), ('Kolkata Knight Riders', 'Deccan Chargers', 0.7777777777777778), ('Deccan Chargers', 'Kolkata Knight Riders', 0.2222222222222222), ('Kolkata Knight Riders', 'Kings XI Punjab', 0.6842105263157895), ('Kings XI Punjab', 'Kolkata Knight Riders', 0.3157894736842105), ('Kolkata Knight Riders', 'Royal Challengers Bangalore', 0.5), ('Royal Challengers Bangalore', 'Kolkata Knight Riders', 0.5), ('Kolkata Knight Riders', 'Delhi Daredevils', 0.5882352941176471), ('Delhi Daredevils', 'Kolkata Knight Riders', 0.4117647058823529), ('Kolkata Knight Riders', 'Kochi Tuskers Kerala', 0.0), ('Kochi Tuskers Kerala', 'Kolkata Knight Riders', 1.0), ('Kolkata Knigh

In [173]:
G=nx.DiGraph()
e=[('a','b',0.7),('b','a',0.3),('a','c',0.8),('c','a',0.2)]
G.add_weighted_edges_from(e)
nx.pagerank(G)

{'a': 0.4864858243244209, 'b': 0.2429732819819368, 'c': 0.2705408936936421}